In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/Predict Accident Risk (Swiss Comp)/Modelling/LGBM

/content/drive/MyDrive/Colab Notebooks/Predict Accident Risk (Swiss Comp)/modelling/lgbm


In [ ]:
!pip install lightgbm

In [ ]:
# Load Necessary Libraries

import os
import pandas as pd
import numpy as np
import copy

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error

seed = 5

# Load and Prepare Saved Data

In [ ]:
# Get parent directory (One level up)
path_parent = os.path.dirname(os.getcwd())
# Join path name as parent directory and file name
path_file_train = os.path.join(path_parent, 'accident_train.csv')
path_file_test = os.path.join(path_parent, 'accident_test.csv')

In [ ]:
## Load preprocessed file

Xy_train = pd.read_csv(path_file_train, sep = ',')
Xy_test = pd.read_csv(path_file_test, sep = ',')

### Prepare Data for Modelling

We will model our data such that we will first predict 'Number_of_Casualties' by using regression or classification methods (Columns 'Accident_ID'and 'postcode' removed during training) and after training, calculate 'Accident Risk Index' using the calculated predicted values, 'Accident Id' and 'postcode'.

Note: We can also model for 'Accident Risk Index' and predict its value directly. However, for that that data needs to be stratified split across 'postcode' as well and all 'postcode' type which contain a unit/single data point/value need to be either upsampled or manually added to training data set (Stratified Split won't work for datasets with unit value counts/frequency)

In [ ]:
def prepare_model_data(data, output_col, remove_col):
  X = data.drop(columns = output_col + remove_col)
  y = data['Number_of_Casualties']
  y = y.values.ravel()
  return X, y

In [ ]:
X_train, y_train = prepare_model_data(Xy_train, ['Number_of_Casualties'], 
                                      ['Accident_ID', 'postcode'])
X_test, y_test = prepare_model_data(Xy_test, ['Number_of_Casualties'], 
                                      ['Accident_ID', 'postcode'])

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(335118, 23) (335118,)
(143623, 23) (143623,)


We will next define a function to prepare the data so as to calculate 'Accident Risk Index'

In [ ]:
def cal_ari(data):
  # we need to calculate Accident_Risk_Index as sum(Number_of_casualities)/count(Accident_ID)
  grouped_train = data.groupby('postcode') # group by postcode
  
  #create aggregation functions
  aggregations = {'Number_of_Casualties': [np.mean],
                  'Accident_ID': [np.count_nonzero]}
  aggregated_ = grouped_train.agg(aggregations)
  
  # formula mentioned above is used to calculate <Accident_Risk_Index>
  aggregated_['Accident_risk_index'] = aggregated_['Number_of_Casualties']['mean']/aggregated_['Accident_ID']['count_nonzero']
  return aggregated_

Preparing Test Data with 'Accident Risk Index'

In [ ]:
ari_test = cal_ari(Xy_test)

# Metric

We will use mean squared error metric to measure performance of our models

## Unbalanced Classes

### Undersampling and Oversampling

Here we will demonstrate how to perform combination of oversampling and downsampling using SMOTE and Tomek-Links from 'imblearn' library to balance out our classes. 

Note: 

  * We can weight out our classes when performing 
  modelling instead of using the above method. 
  
  * We can build separate models and/or model within model in addition to the above methods when appropriate

  * Downsampling leads to loss of data, however
  combination of downsampling and oversampling can 
  give much better results 

In [ ]:
!pip install --upgrade imbalanced-learn

     |████████████████████████████████| 199 kB 5.9 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1


In [ ]:
# Need labels for output if  you want to use custom strategy
from sklearn.preprocessing import LabelEncoder

# combination sampler
from imblearn.combine import SMOTETomek

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

We need to encode out output before we can use our sampler from 'imblearn' library.

In [ ]:
# label encode the target variable

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

Our dataset contains only numeric data types. We can use SMOTE, ADASYN, BorderlineSMOTE etc as oversampling methods. However, it is better to use a combination of oversampling and undersampling to achieve better results.

Here we will use 'SMOTETomek' combination method which uses SMOTE for over-sampling and Tomek links for cleaning.

* Note 1: We can also use sklearn resample to balance out our classes, however we should note that SMOTE method creates synthetic(new) data points.

* Note 2: Random undersampling, although simple and effective, has the drawback of removing data points without any concern for how useful or important they might be in determining the decision boundary between the classes. 'TomekLinks', 'Edited Nearest Neighbours' help in overcoming this drawback.

Create a dictionary to determine in what ratio/amount you want to split

In [ ]:
print("value_counts: {} \nlabels: {}".format(np.bincount(y_train_enc), np.unique(y_train_enc)))

value_counts: [230386  69170  24779   6523   4260] 
labels: [0 1 2 3 4]


We see that the data us heavily unbalanced, and thus won't over-sample a lot so as to balance out the data. We would use sampling in combination with weighing method as oversampling using SMOTE(Synthetic points) in excess might add an unnecessary bias.
Also, note that we cannot control the amount to sample (reduce) with TomekLinks which comes under clean-sampling method and would be done automatically by giving class labels that we want to undersample/reduce.

In [ ]:
# strategy for SMOTE(over-sampling) for output percentage label wise as a dictionary
strategy_smote = {1:150000, 2:75000, 3:25000, 4:25000}
# strategy for tomeklinks(clean-sampling) should be a list of labels 
strategy_tomek = [0]

Create samples using 'SMOTETomek' using the above parameter settings

In [ ]:
comb_sample = SMOTETomek(smote = SMOTE(sampling_strategy = strategy_smote), 
                 tomek = TomekLinks(sampling_strategy = strategy_tomek),
                 random_state = seed)

X_comb, y_comb = comb_sample.fit_resample(X_train, y_train_enc)

In [ ]:
print("New Value Counts: {}".format(np.bincount(y_comb)))

New Value Counts: [202657 150000  75000  25000  25000]


In [ ]:
print("New Distribution Percentage: {}".format((np.round(np.bincount(y_comb)/len(y_comb), 4) * 100)))

New Distribution Percentage: [42.43 31.4  15.7   5.23  5.23]


In [ ]:
# Converting encoding back to labels

y_comb = pd.Series(le.inverse_transform(y_comb), name = 'Number_of_Casualties')

In [ ]:
Xy_train = pd.concat([X_comb, y_comb], axis = 1)

# Training

We will demonstrate now how to use LightGBM algorithm for training and building our model

Here we will use lightgbm classifier for training and evaluating our model.

* Note 1: We can also use lightgbm regressor for training and evaluating our model, however since our output is discrete in nature with only 5 values, lightgbm classifier would be the preferred method. Also, with lightgbm regressor we would be more intereted in its distribution and might have use transformation, resampling techniques, tweedie regressor(weighted regression) to balanced out our data.

* Note 2: Poisson regression is a suitable candiate for modelling data with discrete response variable and we will demonstrate later a Poisson GAM.



### Baseline LightGBM

In [ ]:
model_lgb = LGBMClassifier(class_weight = 'balanced', objective = 'multiclass',
                           random_state = seed)

In [ ]:
lgb = model_lgb.fit(X_train, y_train)

In [ ]:
model_predict = lgb.predict(X_test)

Mean Squared Error for 'Number of Casualties'

In [ ]:
print("Mean Squared Error: {}".format(mean_squared_error(y_test, model_predict,  squared = False)))

Mean Squared Error: 2.070899246319447


Calculating Accident Risk Index for predicted values

In [ ]:
pred_test = copy.deepcopy(Xy_test)

pred_test['Number_of_Casualties'] = model_predict

In [ ]:
ari_pred = cal_ari(pred_test)

Mean Squared Error for 'Accident Risk Index'

In [ ]:
print("Mean Squared Error: {}".format(mean_squared_error(ari_test['Accident_risk_index'], 
                                                         ari_pred['Accident_risk_index'],
                                                         squared = False)))

Mean Squared Error: 1.4931607075321371


### Tuning LGBM Hyperparameters

Now we will start tuning our LGBM classifier model.
Here we will demonstrate how to estimate 'number of trees' and 'learning rate' using Grid Search.
We can fine tune in a similar way as we did with XGBoost.

* Note 1: Optuna is another library useful for the same and gives better results in more efficient way.

* Note 2: We should first focus on feature engineering and trying other models first to check for improvement in score and then fine tune.

In [ ]:
# define evaluation procedure
skf = StratifiedKFold(n_splits = 5, random_state = seed, shuffle = True)

### Tuning Number of Trees/Estimators

In [ ]:
param_grid_custom = {'n_estimators':[100, 250, 500, 1000],
                     'learning_rate':[0.1, 0.5, 0.01]}

In [ ]:
GR = GridSearchCV(estimator = model_lgb, param_grid = param_grid_custom, 
                  scoring = 'neg_mean_squared_error', cv = skf, refit = True, 
                  n_jobs = -1)    
GR_lgbm = GR.fit(X_train, y_train)

print("Best acc score: %f using %s "%(GR_lgbm.best_score_, GR_lgbm.best_params_))

Best acc score: -2.026999 using {'learning_rate': 0.5, 'n_estimators': 1000} 


### Evaluating LGBMBoost

In [ ]:
model_predict = GR_lgbm.predict(X_test)

print("Number of Casualties (Mean Squared Error): {}".format((mean_squared_error(y_test, 
                                                                           model_predict,
                                                                           squared = False))))

Number of Casualties (Mean Squared Error): 1.5078940466662558


Calculating Accident Risk Index for predicted values

In [ ]:
pred_test = copy.deepcopy(Xy_test)

pred_test['Number_of_Casualties'] = model_predict

In [ ]:
ari_pred = cal_ari(pred_test)

In [ ]:
print("Mean Squared Error: {}".format(mean_squared_error(ari_test['Accident_risk_index'], 
                                                         ari_pred['Accident_risk_index'],  
                                                         squared = False)))

Mean Squared Error: 1.0493149452467978
